<a href="https://colab.research.google.com/github/bcmin1018/NLP/blob/main/MT/study/%EA%B8%B0%EA%B3%84%EB%B2%88%EC%97%AD_RNN%2BAttention_%EC%BD%94%EB%93%9C%EB%A6%AC%EB%B7%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference Link
* https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html

In [ ]:
!gdown --id 1DGeE1UY8nAhlYwLIIyNRHJ-SlVaxcURU
!gdown --id 1GPUgAlDKn0Wk1TqILJwzqeS84eBiOngU

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1DGeE1UY8nAhlYwLIIyNRHJ-SlVaxcURU
To: /content/train_en_ko.csv
100% 145k/145k [00:00<00:00, 80.3MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1GPUgAlDKn0Wk1TqILJwzqeS84eBiOngU
To: /content/valid_en_ko.csv
100% 72.5k/72.5k [00:00<00:00, 69.1MB/s]


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 50.4 MB/s 


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim
from nltk.tokenize import word_tokenize
import os
import nltk
import codecs
import csv
from konlpy.tag import Okt
import torch.nn.functional as F
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
train = pd.read_csv("train_en_ko.csv", header = None, names=['english', 'korean'])

In [ ]:
train

,english,korean
0,"Through the snow and sleet and hail, through t...","폭설이 내리고 우박, 진눈깨비가 퍼부어도 눈보라가 몰아쳐도 강풍이 불고 비바람이 휘..."
1,"ever faithful, ever true, nothing stops him, h...",우리의 한결같은 심부름꾼 황새 아저씨 가는 길을 그 누가 막으랴!
2,Look out for Mr Stork That persevering chap,황새 아저씨를 기다리세요
3,He'll come along and drop a bundle in your lap,찾아와 선물을 주실 거예요
4,You may be poor or rich It doesn't matter which,가난하든 부자이든 상관이 없답니다
...,...,...
1995,I'd rather kiss a tarantula!,차라리 독거미와 키스하겠다
1996,-You don't mean that!,못 할 거면서
1997,-l don't--?,못 해?
1998,"Joe, bring me a tarantula. Now listen--","독거미 가져 와 잘 들어, 리나"


# 1. 언어사전 만들기

In [ ]:
# 사전 내 특수 단어의 index를 각각 미리 정의합니다.
SOS_token = 0
EOS_token = 1
UNK_token = 2

class LangDic:
    # 언어마다 단어 사전을 정의합니다.
    def __init__(self, name, tokenizer):
        # 클래스의 첫 시작인 함수입니다. 여기서 모델에 필요한 여러 변수들을 정의합니다.
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "UNK"}
        self.n_words = 2
        self.tokenizer = tokenizer

    def add_sentence(self, sentence):
        # 문장을 받아 문장에서 단어를 확인합니다.
        for word in self.tokenizer(sentence):
            self.add_word(word)

    def add_word(self, word):
        # 단어를 보고 그 단어가 사전에 존재하는지 아닌지를 살펴봅니다.
        # 존재하지 않는 경우 단어를 사전에 등록합니다.
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def sentence2tensor(self, sentence, max_len):
        # 가지고 있는 사전을 바탕으로 문장을 tensor의 형태로 바꿉니다.
        # tensor 내 들어있는 값은 단어 index이며 이를 통해 모델의 임베딩에 입력으로 줄 수 있습니다.
        indexes = list()
        for word in self.tokenizer(sentence):
            try:
                indexes.append(self.word2index[word])
            except KeyError:
                indexes.append(UNK_token)

        indexes.append(EOS_token)
        len_sen = len(indexes)
        if len_sen > max_len:
            indexes = indexes[:max_len-1]
            indexes.append(EOS_token)
            len_sen = max_len

        index_tensor = torch.tensor(indexes)
        return index_tensor, len_sen

In [ ]:
def make_dic(dataset_path):
    # 사전을 만드는 함수입니다.
    data_pairs = load_file(dataset_path)

    # 영어의 경우 NLTK tokenizer를
    # 한국어의 경우 Konlpy 내 Open Korean Text tokenizer를 이용합니다.
    eng_tokenizer = word_tokenize
    kor_tokenizer = Okt().morphs

    eng_dic = LangDic('en', eng_tokenizer)
    kor_dic = LangDic('ko', kor_tokenizer)

    for eng_sen, kor_sen in data_pairs:
        eng_dic.add_sentence(eng_sen)
        kor_dic.add_sentence(kor_sen)

    return eng_dic, kor_dic

def load_file(dataset_path):
    # 데이터를 읽는 함수입니다.
    data_pairs = list()
    # 데이터 파일의 내용을 불러와 영어 문장과 한국어 문장을 모아 리스트에 넣습니다.
    with codecs.open(dataset_path, "r", "utf-8") as csv_f:
        csv_reader = csv.reader(csv_f)
        for one_row in csv_reader:
            data_pairs.append(one_row)

    return data_pairs

In [ ]:
train_file_path = "./train_en_ko.csv"
valid_file_path = "./valid_en_ko.csv"

In [ ]:
eng_dic, kor_dic = make_dic(dataset_path=train_file_path)

# 2. 예제 데이터 샘플
* 먼저 실제 데이터를 준비된 언어 사전을 기반으로 숫자로 치환합니다. 예를 들어 "나는 학교에 갑니다."와 같은 문자로된 데이터가 5, 1, 3 과 같은 숫자로 변형됩니다. 숫자로 변형되는 기준은 언어 사전에 따라 다르게 나타납니다. 치환된 숫자는 학습 모델에 맞게 tensor 형태로 변환됩니다.

In [ ]:
class MTDataset(Dataset):
    # pytorch로 데이터를 불러오기 위해서 Dataset 클래스를 상속받아 새로운 클래스를 만듭니다.
    def __init__(self, data_pairs, eng_dic, kor_dic, max_len):
        super(MTDataset, self).__init__()

        # 데이터를 파일로부터 읽어 이를 전달 받습니다.
        self.max_len = max_len
        self.pair_data = list()

        # 데이터 내 문장을 미리 정의한 사전에 기반하여 tensor로 바꿉니다.
        for eng_sen, kor_sen in data_pairs:
            eng_sen_words, eng_sen_len = eng_dic.sentence2tensor(eng_sen, max_len)
            kor_sen_words, kor_sen_len = kor_dic.sentence2tensor(kor_sen, max_len)
            self.pair_data.append((eng_sen_words, eng_sen_len, kor_sen_words, kor_sen_len))

        self.data_len = len(self.pair_data)

    def __getitem__(self, idx):
        # idx번째 데이터를 반환합니다.
        eng_sen_words, eng_sen_len, kor_sen_words, kor_sen_len = self.pair_data[idx]

        return eng_sen_words, eng_sen_len, kor_sen_words, kor_sen_len

    def __len__(self):
        return self.data_len

In [ ]:
train_tuple = train.to_records(index=False)
for eng_sen, kor_sen in train_tuple:
  eng_dic.add_sentence(eng_sen)
  kor_dic.add_sentence(kor_sen)

In [ ]:
ds = MTDataset(train_tuple, eng_dic, kor_dic, 10)
train_loader = DataLoader(ds, batch_size = 1)

In [ ]:
for input_tensor, input_length, target_tensor, target_length in train_loader:
  input_tensor = input_tensor[0]
  target_tensor = target_tensor[0]
  input_length = input_length
  target_length = target_length
  break

* 최종적으로 입력/출력 텐서값과 길이를 출력하였습니다. 영어를 한국어로 번역하기 때문에 입력은 영어를 뜻하고 출력은 한국어를 의미합니다.

In [ ]:
print("입력 텐서 값 ", input_tensor)
print("출력 텐서 값 ", target_tensor)
print("입력 텐서 길이 ", input_length)
print("출력 텐서 길이 ", target_length)

입력 텐서 값  tensor([2, 3, 4, 5, 6, 5, 7, 8, 9, 1])
출력 텐서 값  tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10,  1])
입력 텐서 길이  tensor([10])
출력 텐서 길이  tensor([10])


# 3. Encoder

Sequence to sequence 모델에 Attention을 적용한다는 의미는 encoder의 입력에 대한 각각의 output, 즉 모든 hidden_state 정보를 이용한다는 것이다. 각 hidden_state를 사용하여 어떤 단어에 집중 할지에 대해 파악한다.

![image](https://user-images.githubusercontent.com/101251439/167293777-526b0adf-24be-4b3c-84b4-6663afbbce76.png)

* Encoder에 필요한 입력 값을 만듭니다.
  * input_size: encoder의 입력 차원 (영어 언어 사전 길이에 해당된다., 한 문자를 원핫 인코딩하면 단어 사전 길이 만큼의 차원을 갖게 되기 때문이다.)



In [ ]:
eng_dic_size = eng_dic.n_words
input_size = eng_dic_size
hidden_size = 256
max_len = 10
print("encoder_input_size: ", input_size)
print("hidden_size: ", hidden_size)
print("max_len :", max_len)

encoder_input_size:  2506
hidden_size:  256
max_len : 10


* Encoder 동작 방식 : 문자 입력 -> 임베딩 -> RNN 순서 -> context vector 및 모든 hidden state 생성
  1. encoder_hidden: 최종 output(context vector)를 담을 배열(텐서) 생성
  2. encoder_outputs: 모든 output을 담을 배열 (텐서) 생성 
  3. embedding layer 생성
  4. RNN layer 생성
  5. 영어 데이터 입력하기
  6. encoder_hidden, encoder_outputs 출력하기

In [ ]:
# encoder의 최종 output인 context vecotor을 담을 3차원 텐서 만들기 간단하게 0으로 만들기
encoder_hidden = torch.zeros(1,1,256)
encoder_hidden.shape

torch.Size([1, 1, 256])

In [ ]:
# encoder의 각 output (hidden_state)을 담을 배열 생성
encoder_outputs = torch.zeros(max_len, hidden_size)
encoder_outputs.shape

torch.Size([10, 256])

* 아래의 그림처럼 hello라는 단어의 중복값을 제거하여 h i e l o 알파뱃으로 언어사전을 만들었다. 이를 각각 원핫인코딩하여 input_size는 5가 된다. 따라서 언어사전 길이가 1000이라고 하면 원핫인코딩하면 한개의 단어를 1000개의 벡터로 표현할 수 있다. 

![image](https://user-images.githubusercontent.com/101251439/167286530-64ebeee5-1419-44b8-8158-75dbee3b7913.png)

In [ ]:
# embedding 층 만들기
encoder_embedding = nn.Embedding(input_size, hidden_size)
encoder_embedding

Embedding(2506, 256)

In [ ]:
# RNN 층 생성
encoder_rnn = nn.GRU(hidden_size, hidden_size)
encoder_rnn

GRU(256, 256)

In [ ]:
# 첫번째 단어만 넣고 임베딩 예시
for idx in range(input_length):
  input_tensor_step = input_tensor[idx]
  print("임베딩할 텐서 값 ", input_tensor_step)
  embedded = encoder_embedding(input_tensor_step).view(1,1,-1)
  # encoder_output, encoder_hidden 생성
  encoder_output, encoder_hidden = encoder_rnn(embedded, encoder_hidden)
  encoder_outputs[idx] = encoder_output[0,0]
  break
print("Output: ", encoder_outputs)
print(encoder_outputs.shape)
print("Hidden: ", encoder_hidden)
print(encoder_hidden.shape)

임베딩할 텐서 값  tensor(2)
Output:  tensor([[-0.1800,  0.1494,  0.0140,  ...,  0.4770,  0.0010, -0.0221],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<CopySlices>)
torch.Size([10, 256])
Hidden:  tensor([[[-0.1800,  0.1494,  0.0140,  0.2764,  0.1389, -0.0278, -0.3010,
           0.1817,  0.2420,  0.1900, -0.0707, -0.3067, -0.0982,  0.2321,
          -0.2731, -0.0454, -0.1016,  0.3294, -0.5924,  0.2126, -0.1507,
           0.3126,  0.2100, -0.1110, -0.0823, -0.3029,  0.1468, -0.0797,
           0.0947,  0.0559,  0.0426, -0.0602,  0.0185,  0.0092, -0.4035,
          -0.2461, -0.0476, -0.2472,  0.0834, -0.2411,  0.2606, -0.0691,
           0.2050, -0.2181, -0.0790,  

encoder를 지난 최종 output (context vector)은 디코더의 첫번째 입력되는 hidden state이다.

![image](https://user-images.githubusercontent.com/101251439/167286811-38f4696b-6f06-4fba-866b-1abf9bd36af5.png)

In [ ]:
# 문장 전체 사용
for idx in range(input_length):
  input_tensor_step = input_tensor[idx]
  print("임베딩할 텐서 값 ", input_tensor_step)
  embedded = encoder_embedding(input_tensor_step).view(1,1,-1)
  # encoder_output, encoder_hidden 생성
  encoder_output, encoder_hidden = encoder_rnn(embedded, encoder_hidden)
  encoder_outputs[idx] = encoder_output[0,0]
  
print("Output: ", encoder_outputs)
print(encoder_outputs.shape)
print("Hidden: ", encoder_hidden)
print(encoder_hidden.shape)

임베딩할 텐서 값  tensor(2)
임베딩할 텐서 값  tensor(3)
임베딩할 텐서 값  tensor(4)
임베딩할 텐서 값  tensor(5)
임베딩할 텐서 값  tensor(6)
임베딩할 텐서 값  tensor(5)
임베딩할 텐서 값  tensor(7)
임베딩할 텐서 값  tensor(8)
임베딩할 텐서 값  tensor(9)
임베딩할 텐서 값  tensor(1)
Output:  tensor([[-0.2101,  0.1963,  0.0205,  ...,  0.6142,  0.0037, -0.0304],
        [-0.3297,  0.0476,  0.2311,  ...,  0.2259, -0.0043,  0.4193],
        [-0.4934, -0.4131, -0.1565,  ..., -0.3029,  0.4336,  0.4013],
        ...,
        [-0.0591,  0.5687, -0.3574,  ...,  0.0246,  0.1674, -0.5449],
        [ 0.1991, -0.0494, -0.3080,  ..., -0.4123,  0.5717, -0.5242],
        [ 0.1855, -0.3274, -0.1775,  ..., -0.4647, -0.0044, -0.4288]],
       grad_fn=<CopySlices>)
torch.Size([10, 256])
Hidden:  tensor([[[ 1.8553e-01, -3.2737e-01, -1.7746e-01,  2.9361e-01, -2.7654e-01,
          -1.6257e-02, -9.3238e-02, -2.9453e-01, -3.6550e-01,  5.1483e-01,
          -4.0575e-01, -2.0769e-01, -1.6290e-01,  2.2070e-02, -2.6099e-01,
          -2.4117e-01, -3.5822e-01,  4.1358e-02, -1.2331e-01, 

# 3. Decoder

디코더 과정에서 인코더의 hidden state를 이용해서 Attention 기법을 사용합니다.
Attention 층을 거친 최종 output은 디코더 입력값과 연산을 통해 다음 단어를 예측합니다.

![image](https://user-images.githubusercontent.com/101251439/167294177-0089f034-8ca5-4185-80d4-4f7ecbc75e28.png)

* 디코더에 필요한 입력값을 만듭니다.
  * input_size & output_size: decoder의 입력 차원 또는 decoder의 출력 차원 (한국어 언어 사전 길이에 해당된다.) 



In [ ]:
kor_dic_size = kor_dic.n_words
output_size = kor_dic_size
hidden_size = 256
print("decoder_hiddenstate: ", hidden_size)
print("decoder_input_size or output_size: ", output_size)

decoder_hiddenstate:  256
decoder_input_size or output_size:  3241


  * Decoder 동작 방식 : SOS 토큰 입력 -> 임배딩 -> dropout -> Attention -> RNN -> output

  1. Decoder 첫 입력값 SOS 토큰 백터 만들기
  2. 디코더 임배딩, 드롭아웃 층 생성
  3. 디코더 RNN 층 생성
  4. 어텐션 층 생성
  


* 디코더에 필요한 층 생성

In [ ]:
#decoder_embedding
decoder_embedding = nn.Embedding(output_size, hidden_size)
decoder_embedding

Embedding(3241, 256)

In [ ]:
#dropout layer
dropout = nn.Dropout(0.1)

In [ ]:
#rnn layer 생성
decoder_rnn = nn.GRU(hidden_size, hidden_size)
decoder_rnn

GRU(256, 256)

* 디코더의 다음 단어를 생성하기 위해 인코더에 입력된 단어 중 어느 단어에 집중을 할까? 집중해야할 단어들에 점수(Attention Score)를 부여하기 위해 위한 어텐션 층을 생성한다.

In [ ]:
#attention layer: 512 길이를 10으로 축소한다.
attn = nn.Linear(hidden_size * 2, max_len)
attn

Linear(in_features=512, out_features=10, bias=True)

In [ ]:
#attention combine
attn_combine = nn.Linear(hidden_size * 2, hidden_size)
attn_combine

Linear(in_features=512, out_features=256, bias=True)

In [ ]:
out = nn.Linear(hidden_size, output_size)

* 디코더 첫번째 SOS 토큰 입력값 만들기

In [ ]:
decoder_input = torch.tensor([[SOS_token]])
decoder_input

tensor([[0]])

* decoder_hidden : 첫번째 hiddens state는 인코더의 최종 ouput

In [ ]:
# decoder에 처음 입력되는 hidden은 encoder에서 출력된 context vector이다.
decoder_hidden = encoder_hidden
decoder_hidden

tensor([[[ 1.8553e-01, -3.2737e-01, -1.7746e-01,  2.9361e-01, -2.7654e-01,
          -1.6257e-02, -9.3238e-02, -2.9453e-01, -3.6550e-01,  5.1483e-01,
          -4.0575e-01, -2.0769e-01, -1.6290e-01,  2.2070e-02, -2.6099e-01,
          -2.4117e-01, -3.5822e-01,  4.1358e-02, -1.2331e-01,  4.3622e-01,
           5.7500e-02, -3.6290e-01, -3.2169e-01,  2.4425e-01, -4.3693e-02,
          -1.2849e-01,  3.4084e-01,  3.0567e-01,  7.5472e-02,  7.5786e-02,
          -1.7097e-02, -2.7780e-02, -1.0886e-01,  7.9470e-02,  7.5075e-02,
          -4.5910e-01,  3.1378e-02,  3.5593e-01, -8.4244e-02, -8.6812e-02,
          -1.5236e-01, -1.3729e-01, -1.2754e-01,  1.9763e-01,  1.7052e-02,
           3.7005e-01,  1.2101e-01,  1.6412e-01, -2.0437e-01, -2.6039e-01,
          -4.8333e-01,  2.3079e-01,  1.1424e-02,  1.4952e-01, -7.5544e-02,
          -4.1887e-01,  2.0382e-01, -3.5353e-01,  1.1993e-01,  3.8428e-01,
           6.6012e-02, -2.4601e-01,  1.0371e-01,  1.9052e-02, -6.6556e-02,
          -4.0570e-01,  9

In [ ]:
# RNN의 첫 입력값인 SOS 토큰을 임베딩 후 dropout을 적용한다.
decoder_embedded = decoder_embedding(decoder_input).view(1,1,-1)
decoder_embedded = dropout(decoder_embedded)
decoder_embedded

tensor([[[ 8.6903e-01,  1.7354e+00,  4.2700e-01, -1.1472e+00, -2.4283e+00,
           9.0323e-02,  1.7649e+00, -4.3337e-02, -2.2300e+00, -5.9895e-01,
          -1.2797e+00, -1.3895e+00, -5.8978e-01,  1.1077e+00, -1.0980e-01,
          -2.7446e+00,  9.1430e-01,  6.2423e-01,  2.4137e+00, -2.2907e-01,
          -6.8633e-02,  2.5096e-02,  1.0036e+00,  0.0000e+00,  5.6463e-01,
          -1.0815e+00, -2.2506e+00, -7.6832e-02,  2.7149e-01,  7.5214e-01,
          -2.3974e-01, -2.4504e-01,  2.8858e+00,  4.2053e-01, -3.7766e-01,
          -6.8074e-01, -1.3725e-01,  4.3047e-01, -1.3843e+00,  2.4304e+00,
           0.0000e+00,  3.4347e-02,  2.7435e-01, -0.0000e+00, -0.0000e+00,
          -2.9426e-02, -1.8375e-01,  5.7472e-01,  6.1659e-01, -1.3018e+00,
          -3.7993e-01, -1.8310e+00,  3.7711e-02, -0.0000e+00,  0.0000e+00,
           1.2380e+00,  2.8910e+00, -9.2709e-02,  1.6688e-01, -6.2623e-01,
           7.0644e-01,  4.0571e-01,  6.2116e-01, -1.6380e-01, -1.9447e-01,
           1.5601e-01, -0

* 어텐션 
디코더 임베딩(256) 값과 인코더의 최종 output(256)을 결합 후 attention layer(512)에 입력한다. 인코더의 최종 output은 전체 문장의 문맥 정보를 갖고 있다. 디코더 입력과의 연산을 통해 다음 단어 출력시 인코더의 어느 부분에 집중할 지에 대해 고려할 수 있다. softmax를 거쳐 어디에 집중을 할지에 대한 비율이 출력된다.

![image](https://user-images.githubusercontent.com/101251439/167294783-c0428b93-485d-418b-b1b1-76499e4ae3c8.png)

In [ ]:
decoder_attention = attn(torch.cat((decoder_embedded[0], decoder_hidden[0]), 1))
decoder_attention

tensor([[-0.3025,  0.5066,  0.0562, -0.1150,  0.4674,  0.2874,  0.5108,  0.0790,
         -0.8283,  0.2255]], grad_fn=<AddmmBackward0>)

In [ ]:
attn_weight = F.softmax(decoder_attention, dim=1)
attn_weight

tensor([[0.0631, 0.1417, 0.0903, 0.0761, 0.1362, 0.1138, 0.1423, 0.0924, 0.0373,
         0.1070]], grad_fn=<SoftmaxBackward0>)

인코더의 각 output에 어텐션을 적용하여 어느 output에 집중할지 계산한다. 예를 들어 "I am a student" 가 있을 때 encoder의 총 output은 4개를 갖게된다. 각 output은 다음과 같은 정보를 갖고 있다. 

1. 첫번째 output: I
2. 두번째 output: I am
3. 세번째 output: I am a 
4. 네번째 output: I am a student

4개의 output 중 어디에 집중을 할지는 attention wight을 곱하여 가장 높은 점수의 output에 집중하게 된다. 인코더의 각 output에 어텐션을 적용하여 attn_applied를 구한다.

In [ ]:
attn_applied = torch.bmm(attn_weight.unsqueeze(0), encoder_outputs.unsqueeze(0))
attn_applied

tensor([[[-0.2038,  0.2284, -0.0172,  0.0805, -0.0537,  0.0781, -0.0471,
          -0.0059,  0.1504,  0.1661, -0.0348, -0.1466,  0.0420,  0.0879,
           0.0175, -0.0273, -0.3623,  0.1270, -0.2731,  0.0968, -0.0994,
          -0.1236, -0.1177, -0.1458, -0.2581, -0.0099,  0.0826, -0.1984,
           0.3341,  0.1162,  0.1301, -0.1599, -0.2528, -0.0453, -0.2783,
          -0.1302,  0.0562,  0.0354,  0.2389, -0.1730,  0.0317, -0.0515,
           0.2313, -0.0363, -0.1642,  0.1484,  0.0613,  0.0418,  0.1846,
          -0.0753, -0.3043,  0.0563, -0.0286, -0.1338, -0.0197, -0.0507,
           0.1117, -0.1336,  0.3326,  0.0676,  0.1393, -0.1731,  0.4065,
           0.0574, -0.0368, -0.0194,  0.2230,  0.1812,  0.1362,  0.1259,
           0.0841, -0.0642, -0.1908,  0.1496,  0.1728,  0.1307, -0.0377,
           0.4715, -0.0162, -0.3498, -0.1698,  0.2358,  0.2808, -0.2156,
          -0.0009, -0.1918,  0.1833,  0.0839, -0.3719, -0.0013, -0.2278,
          -0.0509,  0.0698, -0.1500, -0.0149,  0.01

* 실제 다음 단어를 예측하기 위해 방금 구한 Attention Value와 디코더의 입력값과 결합한다. 

In [ ]:
output = torch.cat((decoder_embedded[0], attn_applied[0]), 1)
output = attn_combine(output).unsqueeze(0)
output = F.relu(output)
output

tensor([[[2.2032e-03, 0.0000e+00, 4.5133e-01, 5.5458e-01, 3.6334e-01,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 5.2375e-01, 4.1301e-01,
          0.0000e+00, 6.3971e-01, 1.2202e-01, 3.6956e-01, 2.3268e-01,
          0.0000e+00, 0.0000e+00, 2.3699e-01, 0.0000e+00, 0.0000e+00,
          3.4387e-01, 7.3438e-02, 0.0000e+00, 0.0000e+00, 7.6014e-01,
          0.0000e+00, 9.4920e-01, 1.0621e+00, 1.6924e-01, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 5.0330e-01, 2.2878e-01, 4.3727e-03,
          0.0000e+00, 7.3372e-01, 6.7497e-01, 5.6222e-01, 3.7617e-01,
          0.0000e+00, 3.4560e-01, 2.4533e-01, 9.6301e-01, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 1.6069e-01, 2.1328e-01,
          4.2347e-01, 0.0000e+00, 4.2865e-01, 0.0000e+00, 9.5506e-02,
          0.0000e+00, 1.9383e-01, 0.0000e+00, 0.0000e+00, 7.9055e-01,
          0.0000e+00, 9.3617e-02, 0.0000e+00, 0.0000e+00, 6.9324e-02,
          0.0000e+00, 0.0000e+00, 1.4140e-01, 0.0000e+00, 4.9590e-01,
          5.9863e-01

* Attention을 거쳐 나온 ouput과 첫번째 sos 토큰의 output을 rnn에 입력한다. 이후 나오는 output은 다음 단어의 백터값을 의미한다.
![image](https://user-images.githubusercontent.com/101251439/167295080-328e0b1f-46c6-48fc-a966-44dec191a27e.png)

In [ ]:
output, decoder_hidden = decoder_rnn(output, decoder_hidden)
decoder_output = F.log_softmax(out(output[0]), dim=1)
_, topi = decoder_output.topk(1)
print(topi)

tensor([[1912]])
